# Aufgabe 1 - Akku-Abenteuer: Tobi's Optimale Routenplanung

Den Code immer nachvollziehbar kommentieren! Bitte beachtet, dass das Notebook von Anfang bis Ende ohne Fehler durchlaufen muss und dass die requirements.txt Datei aktualisiert wird. 

## Teilaufgabe a): lageplan.png laden und verarbeiten

In [ ]:
from PIL import Image
import numpy as np

# open lageplan and downscale it to 21x21 using nearest neighbor interpolation
image = Image.open("lageplan.png").resize((21, 21), Image.NEAREST)
img_array = np.array(image)
# map colors to corresponding labels
color_map = {
    (255, 255, 255, 0): "out",
    (0, 0, 0, 255): "wall",
    (200, 113, 55, 255): "hall",
    (0, 255, 0, 255): "prof",
    (255, 255, 0, 255): "lab",
    (0, 0, 255, 255): "tea"
}
# find all unique colors 
# colors = np.unique(image.reshape(-1, image.shape[2]), axis=0)

# change rgb values to color labels e.g. "out"
mapper = lambda x: color_map.get(tuple(x))
plan = np.apply_along_axis(mapper, 2, img_array)

plan = np.swapaxes(plan, 0, 1)  # flip x and y to make it similar to task


In [ ]:
from PIL import ImageDraw


def visualize_path(path, title):
    """Visualize Path by blending the original image with an overlay"""
    # Create a transparent overlay
    old_image = Image.open("lageplan.png")
    overlay = Image.new("RGBA", old_image.size, (0, 0, 0, 0))
    draw = ImageDraw.Draw(overlay)

    # Draw the path on the overlay
    for x, y in path:
        draw.rectangle([(x*20+5, y*20+5), (x*20+15, y*20+15)], fill=(255, 255, 255, 200))

    # Combine the original image with the overlay
    result = Image.alpha_composite(old_image.convert("RGBA"), overlay)

    result.save("outputs/" + title + ".png")

    # Display the result
    result.show()

## Teilaufgabe b): Breitensuche

In [ ]:
from collections import deque

start = (3, 17)
goal = (1, 3)

def bfs(start, goal):
    """
    Method to execute the breadth first search of a given start and end point
    :return: List of nodes representing the path bfs found
    """
    queue = deque()
    queue.append((start, [])) # append tuple of (node, list of steps taken)

    while queue:
        node, path = queue.popleft()
        
        # break and return path when goal is reached
        if node == goal:
            path.append(goal)
            return path
        
        # one step in each direction (left, right, up, down)
        possible_moves = [(node[0]+1, node[1]), 
                          (node[0]-1, node[1]), 
                          (node[0], node[1]+ 1), 
                          (node[0], node[1]- 1)]
        
        for move in possible_moves:
            if plan[move] != ('wal' or 'out'):
                # only append to queue if move is not labeled 'wal'/'out'
                queue.append((move, path + [node]))
visualize_path(bfs(start, goal), "bfs")

## Teilaufgabe c): A*-Algorithmus

In [ ]:
def heuristic(node, goal):
    """
    :return: The manhattan distance of a given node to goal
    """
    return abs(node[0] - goal[0]) + abs(node[1] - goal[1])

# corresponding costs for a label
cost = {
    "hal": 2,
    "tea": 3,
    "pro": 4,
    "lab": 5,
    "wet": 20,
}

def a_star(start, goal):
    """
    method to execute the a-star search of a given start and end point
    :return: list of nodes representing the path a_star found
    """
    queue = [(start, [], 0)] # append tuple of (node, list of steps taken, path costs accumulated so far)
    visited = np.zeros_like(plan, dtype=int) # array of visited nodes

    # while queue is not empty
    while queue:
        node, path, path_cost = queue.pop(0)
        visited[node] = 1
        
        # break and return path when goal is reached
        if node == goal:
            path.append(goal)
            return path
        
        # one step in each direction (left, right, up, down)
        possible_moves = [(node[0]+1, node[1]), 
                          (node[0]-1, node[1]), 
                          (node[0], node[1]+ 1), 
                          (node[0], node[1]- 1)] 
        
        # filter out moves that are labeled 'wal'/'out' or have already been visited
        possible_moves = filter(lambda x: plan[x] != ('wal' or 'out') and visited[x] == 0, possible_moves) 
        
        # update moves with (current node, path + current node, cost + cost(current label))
        moves = [(move, path + [node], path_cost + cost.get(plan[move], float("inf")))
                 for move in possible_moves]
        queue.extend(moves)
        
        # sort the queue based the lowest value of f(x) = heuristic(x) + accumulated_path_cost(x)
        queue = sorted(queue, key=lambda x:heuristic(x[0], goal) + x[2])

path = a_star(start, goal)
print(path)


## Teilaufgabe d): Greedy Best First Search-Algorithmus

In [ ]:
def gbf(start, goal):
    """
    method to execute the a-star search of a given start and end point
    :return: list of nodes representing the path gbf found
    """
    queue = [(start, [])] # append tuple of (node, list of steps taken)
    visited = np.zeros_like(plan, dtype=int) # array of visited nodes

    # while queue is not empty
    while queue:
        node, path = queue.pop(0)
        visited[node] = 1
        
        # break and return path when goal is reached
        if node == goal:
            path.append(goal)
            return path
        
        # one step in each direction (left, right, up, down)
        possible_moves = [(node[0]+1, node[1]), 
                          (node[0]-1, node[1]), 
                          (node[0], node[1]+ 1), 
                          (node[0], node[1]- 1)]
        
        # filter out moves that are labeled 'wal'/'out' or have already been visited
        possible_moves = filter(lambda x: plan[x] != ('wal' or 'out') and visited[x] == 0, possible_moves)
        
        # extend que with (current node, path + current node)
        queue.extend([(move, path + [node]) for move in possible_moves])
        
        # sort the queue based the lowest value of f(x) = heuristic(x)
        queue = sorted(queue, key=lambda x:heuristic(x[0], goal))

path = gbf(start, goal)
print(path)


## Teilaufgabe e): Dusseliger Doktorand

In [ ]:
plan[3, 3:14] = 'hall'
visualize_path(bfs(start, goal), "bfs")
visualize_path(a_star(start, goal), "a_star")
visualize_path(gbf(start, goal), "gbf")
plan[3, 3:14] = 'wet'
visualize_path(a_star(start, goal), "a_star_wet")
# d) does not have to be executed again because gbf only uses the heuristic